# Sanctions Screening

- **Purpose:** OFAC sanctions screening with fuzzy name matching for fraud detection pipeline  
- **Author:** Devbrew LLC  
- **Last Updated:** November 4, 2025  
- **Status:** In progress  
- **License:** Apache 2.0 (Code) | Public Domain (OFAC Data)

---

## Dataset License Notice

This notebook uses **OFAC Sanctions Lists** (SDN and Consolidated) from the U.S. Department of the Treasury.

**Dataset License:** Public Domain  
- OFAC sanctions data is publicly available from [OFAC Sanctions List Search](https://sanctionslist.ofac.treas.gov/Home)  
- Data can be freely used, redistributed, and incorporated into commercial systems  
- Updates are published regularly; production systems should refresh data periodically  

**Setup Instructions:** See [`../data_catalog/README.md`](../data_catalog/README.md) for download instructions.

**Code License:** This notebook's code is licensed under Apache 2.0 (open source).

**Disclaimer:** This is a research demonstration. Production sanctions screening requires broader list coverage (EU, UN, UK HMT), legal review, and compliance with local regulations.

---

## Notebook Configuration

### Environment Setup

We configure the Python environment with standardized settings, import required libraries for text processing and fuzzy matching, and set a fixed random seed for reproducibility. This ensures consistent results across runs and enables reliable experimentation.

These settings establish the foundation for all sanctions screening operations, including name normalization, tokenization, and similarity scoring.

In [1]:
import warnings
from pathlib import Path
import json
import hashlib
import unicodedata
import re
from typing import Dict, Any, Optional, List, Tuple
import time

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import rapidfuzz as rf
from rapidfuzz import fuzz, process

# Configuration
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
pd.set_option("display.float_format", '{:.2f}'.format)

# Plotting configuration
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 10

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Environment configured successfully")
print(f"pandas: {pd.__version__}")
print(f"numpy: {np.__version__}")
print(f"rapidfuzz: {rf.__version__}")

Environment configured successfully
pandas: 2.3.3
numpy: 2.3.3
rapidfuzz: 3.14.1


### Path Configuration

We define the project directory structure and validate that OFAC data files exist before proceeding. The validation ensures we have the necessary sanctions lists for screening operations.

This configuration pattern ensures we can locate all required data artifacts and provides clear feedback if prerequisites are missing.

In [2]:
# Project paths
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data_catalog"
OFAC_DIR = DATA_DIR / "ofac"
PROCESSED_DIR = DATA_DIR / "processed"
MODELS_DIR = PROJECT_ROOT / "packages" / "models"

# Ensure output directories exist
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Expected OFAC data files
OFAC_FILES = {
    'SDN Primary': OFAC_DIR / 'sdn' / 'sdn.csv',
    'SDN Alternate': OFAC_DIR / 'sdn' / 'alt.csv',
    'SDN Address': OFAC_DIR / 'sdn' / 'add.csv',
    'Consolidated Primary': OFAC_DIR / 'consolidated' / 'cons_prim.csv',
    'Consolidated Alternate': OFAC_DIR / 'consolidated' / 'cons_alt.csv',
    'Consolidated Address': OFAC_DIR / 'consolidated' / 'cons_add.csv',
}

def validate_required_data():
    """Validate that OFAC sanctions data files exist."""
    print("OFAC Data Availability Check:")
    
    all_exist = True
    for name, path in OFAC_FILES.items():
        exists = path.exists()
        status = "Found" if exists else "Missing"
        print(f" - {name:25s}: {status}")
        if not exists:
            all_exist = False
    
    if not all_exist:
        print("\n[WARNING] Some OFAC files are missing; see data_catalog/README.md for instructions")
    else:
        print("\nAll required OFAC data files are available")
    
    return all_exist

data_available = validate_required_data()

OFAC Data Availability Check:
 - SDN Primary              : Found
 - SDN Alternate            : Found
 - SDN Address              : Found
 - Consolidated Primary     : Found
 - Consolidated Alternate   : Found
 - Consolidated Address     : Found

All required OFAC data files are available


## Load & Normalize OFAC Datasets

We load OFAC sanctions lists (SDN and Consolidated) and apply comprehensive text normalization to enable robust fuzzy matching. This step is critical for handling variations in how names appear across different systems and languages.

Our normalization strategy addresses several common challenges in sanctions screening:
- **Unicode variations**: Convert to canonical form (NFKC) to handle different encodings
- **Accent marks**: Strip diacritics to match "José" with "Jose"
- **Case sensitivity**: Lowercase everything for case-insensitive matching
- **Punctuation**: Standardize hyphens, remove quotes that don't affect identity
- **Whitespace**: Collapse multiple spaces to single space

This preprocessing ensures we can match names reliably even when they're formatted differently in transaction data versus sanctions lists.

In [3]:
def normalize_text(text: str) -> str:
    """
    Normalize text for robust fuzzy matching.
    
    Applies NFKC normalization, lowercasing, accent stripping,
    punctuation canonicalization, and whitespace collapse.
    
    Note: Non-Latin scripts (Chinese, Arabic, Cyrillic) are stripped
    because OFAC sanctions lists use romanized names. For example:
    - "中国工商银行" → "" (empty)
    - "INDUSTRIAL AND COMMERCIAL BANK OF CHINA" → "industrial and commercial bank of china"
    
    Args:
        text: Raw text string to normalize
        
    Returns:
        Normalized text string suitable for fuzzy matching.
        Returns empty string if input contains only non-Latin characters.
        
    Examples:
        >>> normalize_text("José María O'Brien")
        'jose maria obrien'
        
        >>> normalize_text("AL-QAIDA")
        'al qaida'
        
        >>> normalize_text("中国工商银行")
        ''
    """
    if not text or pd.isna(text):
        return ""
    
    # Convert to string if not already
    text = str(text)
    
    # Unicode normalization (canonical composition)
    text = unicodedata.normalize("NFKC", text)
    
    # Lowercase
    text = text.lower()
    
    # Strip accent marks (diacritics)
    # Decompose characters, then filter out combining marks
    text = ''.join(
        char for char in unicodedata.normalize("NFD", text)
        if unicodedata.category(char) != 'Mn'
    )
    
    # Remove quotes (single and double)
    text = re.sub(r"['\"]", "", text)
    
    # Replace non-alphanumeric (except space and hyphen) with space
    # Note: This strips non-Latin scripts (Chinese, Arabic, Cyrillic, etc.)
    # OFAC lists use romanized names, so this is intentional behavior
    text = re.sub(r"[^a-z0-9\s-]", " ", text)
    
    # Collapse multiple spaces to single space
    text = re.sub(r"\s+", " ", text)
    
    # Strip leading/trailing whitespace
    text = text.strip()
    
    return text

# Test normalization function
print("Testing text normalization:\n")
test_cases = [
    "José María O'Brien",
    "AL-QAIDA",
    "Société Générale",
    "中国工商银行",  # Chinese - will be stripped (OFAC uses romanized names)
    "  Multiple   Spaces  ",
    "UPPER-case-MiXeD",
]

for test in test_cases:
    normalized = normalize_text(test)
    # Show empty string explicitly for clarity
    display_normalized = f"'{normalized}'" if normalized else "''" 
    print(f"  '{test}' → {display_normalized}")

Testing text normalization:

  'José María O'Brien' → 'jose maria obrien'
  'AL-QAIDA' → 'al-qaida'
  'Société Générale' → 'societe generale'
  '中国工商银行' → ''
  '  Multiple   Spaces  ' → 'multiple spaces'
  'UPPER-case-MiXeD' → 'upper-case-mixed'


### Load OFAC Data Files

We load all OFAC sanctions lists with explicit column mappings since OFAC CSV files don't include headers. We're loading six files total:
- **SDN List**: Primary names, alternate names, addresses
- **Consolidated List**: Primary names, alternate names, addresses

Each sanctions entry can have multiple alternate names (aliases, former names, etc.) and multiple addresses with country information. We'll merge these together to create a comprehensive screening database.

In [5]:
# Define column mappings for OFAC CSV files (they have no headers)
PRIMARY_COLS = [
    'ent_num', 'SDN_Name', 'SDN_Type', 'Program', 'Title',
    'Call_Sign', 'Vess_type', 'Tonnage', 'GRT', 'Vess_flag',
    'Vess_owner', 'Remarks'
]

ALT_COLS = ['ent_num', 'alt_num', 'alt_type', 'alt_name', 'alt_remarks']

ADD_COLS = [
    'ent_num', 'Add_num', 'Address', 'City_State_Province',
    'Country', 'Add_Remarks'
]

print("Loading OFAC Sanctions Lists...\n")

# Load SDN (Specially Designated Nationals) List
print("Loading SDN List...")
sdn_primary = pd.read_csv(
    OFAC_DIR / 'sdn' / 'sdn.csv',
    header=None,
    names=PRIMARY_COLS,
    dtype={'ent_num': str},
    encoding='utf-8'
)

sdn_alt = pd.read_csv(
    OFAC_DIR / 'sdn' / 'alt.csv',
    header=None,
    names=ALT_COLS,
    dtype={'ent_num': str, 'alt_num': str},
    encoding='utf-8'
)

sdn_add = pd.read_csv(
    OFAC_DIR / 'sdn' / 'add.csv',
    header=None,
    names=ADD_COLS,
    dtype={'ent_num': str, 'Add_num': str},
    encoding='utf-8'
)

print(f" - Primary entities: {len(sdn_primary):,}")
print(f" - Alternate names:  {len(sdn_alt):,}")
print(f" - Addresses:        {len(sdn_add):,}")

# Load Consolidated List
print("\nLoading Consolidated List...")
cons_primary = pd.read_csv(
    OFAC_DIR / 'consolidated' / 'cons_prim.csv',
    header=None,
    names=PRIMARY_COLS,
    dtype={'ent_num': str},
    encoding='utf-8'
)

cons_alt = pd.read_csv(
    OFAC_DIR / 'consolidated' / 'cons_alt.csv',
    header=None,
    names=ALT_COLS,
    dtype={'ent_num': str, 'alt_num': str},
    encoding='utf-8'
)

cons_add = pd.read_csv(
    OFAC_DIR / 'consolidated' / 'cons_add.csv',
    header=None,
    names=ADD_COLS,
    dtype={'ent_num': str, 'Add_num': str},
    encoding='utf-8'
)

print(f" - Primary entities: {len(cons_primary):,}")
print(f" - Alternate names:  {len(cons_alt):,}")
print(f" - Addresses:        {len(cons_add):,}")

print("\nAll OFAC files loaded successfully")

Loading OFAC Sanctions Lists...

Loading SDN List...
 - Primary entities: 17,945
 - Alternate names:  19,898
 - Addresses:        23,628

Loading Consolidated List...
 - Primary entities: 444
 - Alternate names:  1,067
 - Addresses:        573

All OFAC files loaded successfully


### Consolidate Names and Normalize

We merge primary names with their alternate names (aliases, former names) and create a unified sanctions database. Each row will represent a distinct name associated with a sanctioned entity, including both the official name and all known aliases.

We also extract country information from address records to enable geographic filtering during screening. This is important because many sanctions programs are country-specific.

In [7]:
def build_sanctions_index(
    primary_df: pd.DataFrame,
    alt_df: pd.DataFrame,
    add_df: pd.DataFrame,
    source_name: str
) -> pd.DataFrame:
    """
    Build unified sanctions index from primary, alternate, and address files.
    
    Args:
        primary_df: Primary sanctions entities
        alt_df: Alternate names (aliases)
        add_df: Address records with country info
        source_name: Source identifier ('SDN' or 'Consolidated')
        
    Returns:
        DataFrame with columns: uid, name, name_norm, name_type, entity_type, 
                                program, country, remarks, source
    """
    print(f"\nBuilding {source_name} sanctions index...")
    
    # Process primary names
    primary_records = []
    for _, row in primary_df.iterrows():
        primary_records.append({
            'uid': f"{source_name}_{row['ent_num']}",
            'ent_num': row['ent_num'],
            'name': row['SDN_Name'],
            'name_type': 'primary',
            'entity_type': row['SDN_Type'],
            'program': row['Program'],
            'remarks': row['Remarks'],
            'source': source_name
        })
    
    # Process alternate names
    alt_records = []
    for _, row in alt_df.iterrows():
        alt_records.append({
            'uid': f"{source_name}_{row['ent_num']}_alt_{row['alt_num']}",
            'ent_num': row['ent_num'],
            'name': row['alt_name'],
            'name_type': row['alt_type'],  # aka, fka, nka
            'entity_type': None,  # Will be filled from primary
            'program': None,      # Will be filled from primary
            'remarks': row['alt_remarks'],
            'source': source_name
        })
    
    # Combine primary and alternate names
    all_names = pd.DataFrame(primary_records + alt_records)
    
    # Fill entity_type and program from primary records for alternates
    entity_info = primary_df[['ent_num', 'SDN_Type', 'Program']].copy()
    entity_info.columns = ['ent_num', 'entity_type_fill', 'program_fill']
    
    all_names = all_names.merge(entity_info, on='ent_num', how='left')
    all_names['entity_type'] = all_names['entity_type'].fillna(all_names['entity_type_fill'])
    all_names['program'] = all_names['program'].fillna(all_names['program_fill'])
    all_names.drop(columns=['entity_type_fill', 'program_fill'], inplace=True)
    
    # Extract country information from addresses (take first country per entity)
    if len(add_df) > 0:
        country_map = add_df.groupby('ent_num')['Country'].first().to_dict()
        all_names['country'] = all_names['ent_num'].map(country_map)
    else:
        all_names['country'] = None
    
    # Apply text normalization
    print("  Normalizing names...")
    all_names['name_norm'] = all_names['name'].apply(normalize_text)
    
    # Remove records with empty normalized names
    before_count = len(all_names)
    all_names = all_names[all_names['name_norm'].str.len() > 0].copy()
    after_count = len(all_names)
    
    if before_count > after_count:
        print(f"  Removed {before_count - after_count} records with empty normalized names")
    
    # Reorder columns
    columns = [
        'uid', 'ent_num', 'name', 'name_norm', 'name_type', 
        'entity_type', 'program', 'country', 'remarks', 'source'
    ]
    all_names = all_names[columns]
    
    print(f"Created {len(all_names):,} name records")
    
    return all_names

# Build indices for both lists
sdn_index = build_sanctions_index(sdn_primary, sdn_alt, sdn_add, 'SDN')
cons_index = build_sanctions_index(cons_primary, cons_alt, cons_add, 'Consolidated')

# Combine into single index
sanctions_index = pd.concat([sdn_index, cons_index], ignore_index=True)

print(f"\nCombined Sanctions Index Summary:")
print(f" - Total name records: {len(sanctions_index):,}")
print(f" - From SDN:           {len(sdn_index):,}")
print(f" - From Consolidated:  {len(cons_index):,}")
print(f" - Unique entities:    {sanctions_index['ent_num'].nunique():,}")


Building SDN sanctions index...
  Normalizing names...
  Removed 2 records with empty normalized names
Created 37,841 name records

Building Consolidated sanctions index...
  Normalizing names...
  Removed 2 records with empty normalized names
Created 1,509 name records

Combined Sanctions Index Summary:
 - Total name records: 39,350
 - From SDN:           37,841
 - From Consolidated:  1,509
 - Unique entities:    18,310


### Validation Checks

We perform data quality validation to ensure our sanctions index is ready for fuzzy matching:
1. **Non-empty canonical names**: Every record must have valid normalized text
2. **Unique UIDs**: Each name record has a globally unique identifier
3. **Field completeness**: Key fields (entity_type, program) are populated
4. **Normalization quality**: Check sample names to verify normalization worked correctly

These checks catch data quality issues before they cause problems in production screening.

In [10]:
# Validation Check 1: Non-empty canonical names
empty_names = sanctions_index[sanctions_index['name_norm'].str.len() == 0]
print(f"Validation Check 1: Non-empty canonical names")
print(f" - Empty normalized names: {len(empty_names)}")
assert len(empty_names) == 0, "Found records with empty normalized names!"
print(f"PASS - All records have valid normalized names\n")

# Validation Check 2: Unique UIDs
print(f"Validation Check 2: Unique UIDs")
duplicate_uids = sanctions_index['uid'].duplicated().sum()
print(f" - Duplicate UIDs: {duplicate_uids}")
assert duplicate_uids == 0, "Found duplicate UIDs!"
print(f"PASS - All UIDs are unique\n")

# Validation Check 3: Field completeness
print(f"Validation Check 3: Field completeness")
print(f" - Records with entity_type: {sanctions_index['entity_type'].notna().sum():,} / {len(sanctions_index):,}")
print(f" - Records with program:     {sanctions_index['program'].notna().sum():,} / {len(sanctions_index):,}")
print(f" - Records with country:     {sanctions_index['country'].notna().sum():,} / {len(sanctions_index):,}")

# Country is optional (not all entities have addresses)
entity_type_coverage = sanctions_index['entity_type'].notna().mean()
program_coverage = sanctions_index['program'].notna().mean()

if entity_type_coverage < 0.95:
    print(f"[WARNING] Entity type coverage is low: {entity_type_coverage*100:.1f}%")
if program_coverage < 0.95:
    print(f"[WARNING] Program coverage is low: {program_coverage*100:.1f}%")

print(f"PASS - Key fields adequately populated\n")

# Validation Check 4: Sample normalization quality
print(f"Validation Check 4: Sample normalization quality")
print(f"Checking 10 random samples...")

sample_indices = np.random.choice(len(sanctions_index), size=10, replace=False)
for idx in sample_indices:
    row = sanctions_index.iloc[idx]
    original = row['name']
    normalized = row['name_norm']
    print(f" - '{original}' → '{normalized}'")

Validation Check 1: Non-empty canonical names
 - Empty normalized names: 0
PASS - All records have valid normalized names

Validation Check 2: Unique UIDs
 - Duplicate UIDs: 0
PASS - All UIDs are unique

Validation Check 3: Field completeness
 - Records with entity_type: 39,350 / 39,350
 - Records with program:     39,350 / 39,350
 - Records with country:     39,350 / 39,350
PASS - Key fields adequately populated

Validation Check 4: Sample normalization quality
Checking 10 random samples...
 - 'AL-HARAMAYN HUMANITARIAN FOUNDATION' → 'al-haramayn humanitarian foundation'
 - 'AKTSIONERNOE OBSHCHESTVO RT-STROITELNYE TEKHNOLOGII' → 'aktsionernoe obshchestvo rt-stroitelnye tekhnologii'
 - 'VALLE VALLE, Luis Alonso' → 'valle valle luis alonso'
 - 'GLOBAL SEA LINE CO LTD' → 'global sea line co ltd'
 - 'TED TEKNOLOJI' → 'ted teknoloji'
 - 'CLOSED JOINT STOCK COMPANY 'IFD KAPITAL'' → 'closed joint stock company ifd kapital'
 - 'JOINT STOCK COMPANY SEVERGAZBANK' → 'joint stock company severgazb

### Analyze Sanctions Index

We examine the distribution of entity types, programs, and countries in our sanctions database. This helps us understand what we're screening against and can inform filtering strategies during production deployment.

In [18]:
# Distribution analysis
print("Entity Type Distribution:")
entity_type_dist = sanctions_index['entity_type'].value_counts()
for entity_type, count in entity_type_dist.head(10).items():
    pct = (count / len(sanctions_index)) * 100
    print(f"{str(entity_type)[:30]:30s}: {count:>6,} ({pct:>5.1f}%)")

print("\nSanctions Program Distribution (Top 15):")
program_dist = sanctions_index['program'].value_counts()
for program, count in program_dist.head(15).items():
    pct = (count / len(sanctions_index)) * 100
    program_str = str(program)[:50] if pd.notna(program) else "Unknown"
    print(f"{program_str:40s}: {count:>6,} ({pct:>5.1f}%)")

print("\nCountry Distribution (Top 15):")
country_dist = sanctions_index['country'].value_counts()
for country, count in country_dist.head(15).items():
    pct = (count / len(sanctions_index)) * 100
    country_str = str(country)[:30] if pd.notna(country) else "Unknown"
    print(f"{country_str:30s}: {count:>6,} ({pct:>5.1f}%)")

# Name type distribution
print("\nName Type Distribution:")
name_type_dist = sanctions_index['name_type'].value_counts()
for name_type, count in name_type_dist.items():
    pct = (count / len(sanctions_index)) * 100
    print(f"{str(name_type):30s}: {count:>6,} ({pct:>5.1f}%)")

Entity Type Distribution:
-0-                           : 21,308 ( 54.1%)
individual                    : 16,149 ( 41.0%)
vessel                        :  1,555 (  4.0%)
aircraft                      :    338 (  0.9%)

Sanctions Program Distribution (Top 15):
RUSSIA-EO14024                          : 10,339 ( 26.3%)
SDGT                                    :  7,037 ( 17.9%)
SDNTK                                   :  2,395 (  6.1%)
UKRAINE-EO13662] [RUSSIA-EO14024        :  1,415 (  3.6%)
GLOMAG                                  :  1,218 (  3.1%)
NPWMD] [IFSR                            :  1,122 (  2.9%)
IRAN                                    :    837 (  2.1%)
UKRAINE-EO13662                         :    785 (  2.0%)
BELARUS-EO14038                         :    642 (  1.6%)
SDGT] [IFSR                             :    622 (  1.6%)
IRAN-EO13902                            :    572 (  1.5%)
IRAN-EO13846                            :    553 (  1.4%)
PAARSSR-EO13894                         :   

### Save Normalized Sanctions Index

We save the normalized sanctions index as the foundation for our fuzzy matching pipeline. This database contains all sanctioned entity names with proper text normalization, metadata enrichment, and quality validation applied.

The artifacts enable fast loading and consistent screening across the fraud detection system.

In [19]:
# Save normalized sanctions index
sanctions_index_path = MODELS_DIR / "sanctions_index.parquet"
sanctions_index.to_parquet(sanctions_index_path, index=False)

print(f"Saved sanctions index: {sanctions_index_path}")
print(f" - Shape: {sanctions_index.shape}")
print(f" - Size: {sanctions_index_path.stat().st_size / 1024:.1f} KB")

# Save metadata for pipeline tracking
metadata = {
    "created_at": pd.Timestamp.now().isoformat(),
    "total_records": len(sanctions_index),
    "unique_entities": sanctions_index['ent_num'].nunique(),
    "sources": sanctions_index['source'].value_counts().to_dict(),
    "entity_types": entity_type_dist.head(10).to_dict(),
    "top_programs": program_dist.head(10).to_dict(),
    "top_countries": country_dist.head(10).to_dict(),
    "name_types": name_type_dist.to_dict(),
    "country_coverage_pct": float(sanctions_index['country'].notna().mean() * 100),
    "validation": {
        "empty_normalized_names": 0,
        "duplicate_uids": 0,
        "entity_type_coverage_pct": float(entity_type_coverage * 100),
        "program_coverage_pct": float(program_coverage * 100)
    }
}

metadata_path = MODELS_DIR / "sanctions_index_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Saved metadata: {metadata_path}")
print(f"Sanctions Index Ready: {len(sanctions_index):,} normalized name records")

Saved sanctions index: /Users/joekariuki/Documents/Devbrew/research/devbrew-payments-fraud-sanctions/packages/models/sanctions_index.parquet
 - Shape: (39350, 10)
 - Size: 2874.1 KB
Saved metadata: /Users/joekariuki/Documents/Devbrew/research/devbrew-payments-fraud-sanctions/packages/models/sanctions_index_metadata.json
Sanctions Index Ready: 39,350 normalized name records


## Tokenization & Canonical Forms

To enable efficient fuzzy matching, we tokenize normalized names and create canonical representations optimized for different similarity algorithms. This approach improves matching accuracy by:
- **Removing noise**: Filtering out common business suffixes (Ltd, Inc, LLC) and honorifics (Mr, Mrs)
- **Token-based matching**: Breaking names into words for flexible comparison
- **Sorted tokens**: Enabling order-independent matching (e.g., "John Doe" matches "Doe John")
- **Token sets**: Creating unique word bags for set-based similarity

These canonical forms serve as inputs to RapidFuzz's token_sort_ratio and token_set_ratio algorithms, which are robust to word order variations and common name formatting differences.

In [22]:
# Define stopwords for name tokenization
# These are common business/legal terms and honorifics that add noise to matching
STOPWORDS = {
    # Business suffixes
    "ltd", "inc", "llc", "co", "corp", "corporation", "company",
    "sa", "gmbh", "ag", "nv", "bv", "plc", "limited",
    # Honorifics
    "mr", "mrs", "ms", "dr", "prof",
    # Common words
    "the", "of", "and", "for", "de", "la", "el"
}

def tokenize(name: str) -> List[str]:
    """
    Tokenize a normalized name into words, filtering stopwords and short tokens.
    
    Splits on whitespace and hyphens, removes tokens shorter than 2 characters,
    and filters out common business/legal terms that don't aid matching.
    
    Args:
        name: Normalized name string (already lowercased and cleaned)
        
    Returns:
        List of filtered tokens
        
    Examples:
        >>> tokenize("john doe")
        ['john', 'doe']
        
        >>> tokenize("acme corporation ltd")
        ['acme']
        
        >>> tokenize("al-qaida")
        ['al', 'qaida']
    """
    if not name:
        return []
    
    # Split on whitespace and hyphens
    tokens = [t for t in re.split(r'[\s-]+', name) if t]
    
    # Filter: length >= 2 and not in stopwords
    filtered = [t for t in tokens if len(t) >= 2 and t not in STOPWORDS]
    
    return filtered

# Test tokenization function
print("Testing tokenization:\n")
test_names = [
    "john doe",
    "acme corporation ltd",
    "al-qaida",
    "banco nacional de cuba",
    "mr jose maria obrien",
    "china telecom co ltd"
]

for name in test_names:
    tokens = tokenize(name)
    print(f"  '{name}' → {tokens}")

Testing tokenization:

  'john doe' → ['john', 'doe']
  'acme corporation ltd' → ['acme']
  'al-qaida' → ['al', 'qaida']
  'banco nacional de cuba' → ['banco', 'nacional', 'cuba']
  'mr jose maria obrien' → ['jose', 'maria', 'obrien']
  'china telecom co ltd' → ['china', 'telecom']
